In [ ]:
# header / imports
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sc3nb as scn
from sc3nb import Synth

In [ ]:
sc = scn.startup()

# Nodes (Synth and Group)

One of the most important objects in SuperCollider are Nodes.

In [ ]:
sc.server.dump_tree()

In [ ]:
sc.server.query_all_nodes()

In [ ]:
help(scn.AddAction)

A node is either a [Synth](#Synth) or a [Group](#Group) of nodes

## Synth

### Create and control a Synth

Create a new Synth

In [ ]:
synth = Synth(name="default", args={"freq": 100})
synth

Free a synth

In [ ]:
synth.free()

Start the synth again

In [ ]:
synth.new()

Calling `new` on an already running synth will cause a SuperCollider Server error

In [ ]:
synth.new()

Pause a synth

In [ ]:
synth.run(False)

Run a paused synth

In [ ]:
synth.run() # default flag for run is True

### Set / get Synth parameters

Set synth parameters, using any for the following `set` calls

    set(key, value, ...)
    set(list_of_keys_and_values])
    set(dict)

In [ ]:
synth.set("freq", 200)

In [ ]:
synth.set(["freq", 30, "amp", 0.3])

In [ ]:
synth.set({"freq": 130, "amp": 0.1})

The Synth does save its current arguments in `current_args`

In [ ]:
synth.current_args

However these are only cached values on the python object. 
Updating them will only affect the `new` call. See below why this can be useful.

In [ ]:
synth.current_args['freq'] = 440
synth

In [ ]:
synth.free()
synth.new()

To see what arguments can be set you can look at the `synth_desc`

In [ ]:
synth.synth_desc

You can use ``get`` to see the current value. This will request the current value from the SuperCollider audio server

In [ ]:
synth.get("freq")

In [ ]:
synth.get("pan")

This will also update the `current_args`

In [ ]:
synth

This is also possible directly with

In [ ]:
synth.pan

Which can also be used for setting the argument

In [ ]:
synth.pan = -1 

You can also query information about the Synth. Look at [Group](#Group) for more information about these values

In [ ]:
synth.query()

In [ ]:
synth.free()

Keep in mind that getting a synth value is always querying the server.

This means we need to receive a message which cant be done using the Bundler.
If you want to use relative values in a Bundler you should use the cached values from `current_args`

In [ ]:
with sc.server.bundler() as bundle:
    synth.new({"freq": 600})
    for _ in range(100):
        synth.set(['freq', synth.current_args['freq'] * 0.99])
        bundle.wait(0.05)
    synth.free()

Some methods also allow getting the OSC Message instead of sending

In [ ]:
synth.new(return_msg=True)

Refer to the osc communication example notebook if you want to learn more about messages and bundles.

## Group

In [ ]:
sc.server.default_group

In [ ]:
g0 = scn.Group()

In [ ]:
g1 = scn.Group(target=g0)

In [ ]:
sc.server.query_all_nodes()

In [ ]:
g0.query()

In [ ]:
g1.query()

In [ ]:
s0 = scn.Synth(target=g0, args={"freq": 200})

In [ ]:
s1 = scn.Synth(target=g1, args={"freq": 600})

In [ ]:
s2 = scn.Synth(target=g1, args={"freq": 1200})

In [ ]:
g0.dump_tree()

In [ ]:
sc.server.query_all_nodes()

In [ ]:
g0

In [ ]:
g0.run(False)

In [ ]:
g0.run(True)

In [ ]:
g1.run(False)

In [ ]:
s0.freq, s1.freq, s2.freq

In [ ]:
g0.set("freq", 100)
s0.freq, s1.freq, s2.freq

In [ ]:
sc.server.dump_tree()

In [ ]:
g1.run(True)

In [ ]:
sc.server.query_all_nodes()

In [ ]:
g1.set("freq", 440)

s0.freq, s1.freq, s2.freq

In [ ]:
g0.dump_tree()
g0.move_node_to_tail(s1)
g0.dump_tree()

In [ ]:
g1.set("freq", 800)

s0.freq, s1.freq, s2.freq

In [ ]:
sc.server.default_group.dump_tree(True)  # see console

In [ ]:
sc.server.free_all()
sc.server.dump_tree()

In [ ]:
sc.exit()